# NOTTINGHAM FOREST v. ASTON VILLA - match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [53]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""

!!! INPUT TEAMS HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

"""
home_side = "Nottingham Forest"
away_side = "Aston Villa"
matchday = "2024-12-14"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Nottingham Forest v. Aston Villa


### Calculate Projected Goals

In [54]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.1428571428571428 0.8571428571428571
1.4285714285714286 2.0
1.63 1.38
0.7011393514461 1.2269938650306749 1.0351966873706004 0.6211180124223603

 ================== projected_goals ==================== 

HOME Nottingham Forest projected goals: 1.4022787028922
AWAY Aston Villa projected goals:  0.8873114463176576


### Poisson Distribution of Goal Probabilities

In [55]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Nottingham Forest v. Aston Villa [[1.01307974e-01 1.42062015e-01 9.96052691e-02 4.65581159e-02
  1.63218636e-02 4.57756034e-03 1.06983590e-03 2.14315442e-04
  3.75662474e-05]
 [8.98917254e-02 1.26053252e-01 8.83808954e-02 4.13115491e-02
  1.44825764e-02 4.06172168e-03 9.49277636e-04 1.90164545e-04
  3.33329614e-05]
 [3.98809784e-02 5.59242467e-02 3.92106901e-02 1.83281052e-02
  6.42527790e-03 1.80200607e-03 4.21152456e-04 8.43675885e-05
  1.47883591e-05]
 [1.17956162e-02 1.65407414e-02 1.15973647e-02 5.42091251e-03
  1.90040754e-03 5.32980204e-04 1.24564465e-04 2.49534423e-05
  4.37396009e-06]
 [2.61659632e-03 3.66919730e-03 2.57261861e-03 1.20250943e-03
  4.21563341e-04 1.18229859e-04 2.76318689e-05 5.53536875e-06
  9.70266214e-07]
 [4.64347173e-04 6.51144152e-04 4.56542788e-04 2.13400076e-04
  7.48115956e-05 2.09813414e-05 4.90361471e-06 9.82319211e-07
  1.72185664e-07]
 [6.86700936e-05 9.62946099e-05 6.75159403e-05 3.15587217e-05
  1.10635308e-05 3.10283074e-06 7.25172243e-07 1.4527

### Under Market

In [56]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Nottingham Forest v. Aston Villa
    Goals      Prob  Implied Odds
0    0.5  0.101308      9.870891
1    1.5  0.333262      3.000645
2    2.5  0.598801      1.670003
3    3.5  0.801460      1.247723


In [57]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Nottingham Forest v. Aston Villa
    Goals      Prob  Implied Odds
0    0.5  0.898675      1.112749
1    1.5  0.666721      1.499877
2    2.5  0.401182      2.492635
3    3.5  0.198523      5.037200


### 1 x 2 Probability

In [58]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                   winner      prob  implied_odds
0  Nottingham Forest (1)  0.489678      2.042160
1               Draw (x)  0.272436      3.670585
2        Aston Villa (2)  0.237869      4.203989


# Full Results Summary:

In [59]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):
    translations = {
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City"
    }
    if string in translations:
        return translations["string"]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
filtered_odds = odds_df.loc[odds_df["match"] == match_string]
implied = {
    "1": 1 / filtered_odds["1"],
    "x": 1 / filtered_odds["x"],
    "2": 1 / filtered_odds["2"],
    "point": filtered_odds["ovr_und_point"],
    "over": 1 / filtered_odds["over"],
    "under": 1 / filtered_odds["under"]
}
implied_odds_df = pd.DataFrame(implied)

print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)

CALCULATED PROBABILITIES FOR Nottingham Forest v. Aston Villa


        ~ ~ moneyline market ~ ~

                   winner      prob  implied_odds
0  Nottingham Forest (1)  0.489678      2.042160
1               Draw (x)  0.272436      3.670585
2        Aston Villa (2)  0.237869      4.203989


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.898675      1.112749
1    1.5  0.666721      1.499877
2    2.5  0.401182      2.492635
3    3.5  0.198523      5.037200


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.101308      9.870891
1    1.5  0.333262      3.000645
2    2.5  0.598801      1.670003
3    3.5  0.801460      1.247723

 ~~ MyBookie Odds for Nottingham Forest v. Aston Villa ~~
       1    x     2  ovr_und_point  over  under
4  2.59  3.3  2.63            2.5  1.85   1.89

 ~~ MyBookie Odds IMPLIED for Nottingham Forest v. Aston Villa ~~
         1        x         2  point      over     under
4  0.3861  0.30303  0.3802

# Conclusion and Pick

MB is not terribly confident on the over under, giving a implied 54% and 53% for over and under respectfully. I don't want to enter a coin toss for 1.8 odds so I'm thinking money line.

RISKIER PICK: Nottingham Forest to win at home. GUTS. This is a scary pick to me, but it's what makes the game fun. Valued at 2.59 by MB - implied 38% of hitting. We have it at 48%. We could pad it with a 1 x or a draw no bet buuuuut... I think *it's worth a fucking punt eh lads?*

### My Pick:

**Nottingham Forest @ 2.59**